In [1]:
import numpy as np
from pydrake.all import (AbstractValue, AddMultibodyPlantSceneGraph, AngleAxis, Box, 
                         Capsule, ContactResults, Concatenate, DiagramBuilder, LeafSystem,
                         InverseDynamicsController, LoadModelDirectives, MeshcatVisualizer, Parser,
                         PiecewisePolynomial, PiecewisePose, PointCloud, TrajectorySource, 
                         ProcessModelDirectives, RandomGenerator, BasicVector, SceneGraph, 
                         RigidTransform, RollPitchYaw, RotationMatrix, Simulator, StartMeshcat,
                         UniformlyRandomRotationMatrix, Rgba, ContactResults, BaseField, Fields,
                         ConstantVectorSource, FindResourceOrThrow, Integrator, JacobianWrtVariable, 
                         MeshcatVisualizerParams, MultibodyPlant, MultibodyPositionToGeometryPose,
                         Quaternion, JointSliders, Rgba, Sphere, StateInterpolatorWithDiscreteDerivative)
from pydrake.common import FindResourceOrThrow, temp_directory
from pydrake.geometry import (
    MeshcatVisualizer,
    MeshcatVisualizerParams,
    Role,
    StartMeshcat,
)
from pydrake.math import RigidTransform, RollPitchYaw
from pydrake.multibody.meshcat import JointSliders
from pydrake.multibody.parsing import Parser
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.systems.analysis import Simulator
from pydrake.systems.framework import DiagramBuilder

from manipulation import FindResource, running_as_notebook
from manipulation.station import MakeHardwareStation, load_scenario
from manipulation.clutter import GenerateAntipodalGraspCandidate
from manipulation.meshcat_utils import AddMeshcatTriad
from manipulation.pick import (MakeGripperCommandTrajectory, MakeGripperFrames,
                               MakeGripperPoseTrajectory)
from manipulation.scenarios import (AddIiwaDifferentialIK, 
                                    MakeManipulationStation, ycb, AddRgbdSensors)

In [ ]:
#Creating small leaf system that takes a trajectory and provides one pose at a time. 
class PoseSystem(LeafSystem):
    def __init__(self, plant, traj):
        LeafSystem.__init__(self)
        port = self.DeclareAbstractOutputPort("pose", lambda: AbstractValue.Make(RigidTransform()), self.DoCalcOutput)
        self.traj = traj 
    
    def DoCalcOutput(self, context, output):
        """Constructs the output pose based on the trajectory that was passed in."""
        output.set_value(self.traj.GetPose(context.get_time()))

In [ ]:
#Painter Class
class IIWA_Painter():
    def __init__(self, traj1=None, traj2 = None):
        meshcat.Delete()
        meshcat.DeleteAddedControls()
        builder = DiagramBuilder()
        
        # set up the system of manipulation stationrio
        scenario = load_scenario(data=sketchbot_directives)
        self.station = MakeHardwareStation(scenario, meshcat=meshcat)
        builder.AddSystem(self.station)
        self.plant = self.station.GetSubsystemByName("plant")
        # Export the camera outputs
        builder.ExportOutput(
            self.station.GetOutputPort("camera.rgb_image"), "rgb_image"
        )
        builder.ExportOutput(
            self.station.GetOutputPort("camera.depth_image"), "depth_image"
        )
        if traj1 is not None:
            """ This is the trajectory for the iiwa arm """
            ## DO NOT DELETE ###### PSEUDOINVERSE CONTROLLER ##############  
            # traj1_V_G = traj1.MakeDerivative()
            # V_G_source1 = builder.AddSystem(TrajectorySource(traj1_V_G))
            

            # self.controller = builder.AddSystem(PseudoInverseController(self.plant))
            # builder.Connect(V_G_source1.get_output_port(), self.controller.GetInputPort("V_G"))

            # self.integrator = builder.AddSystem(Integrator(7))
            # builder.Connect(self.controller.get_output_port(), self.integrator.get_input_port())
            # builder.Connect(self.integrator.get_output_port(), self.station.GetInputPort("iiwa_position"))
            # builder.Connect(self.station.GetOutputPort("iiwa_position_measured"), self.controller.GetInputPort("iiwa_position"))
            # ##############################################


            # ### DO NOT DELETE ##### Diff IK CONTROLLER ############## 
            pose_picker = builder.AddSystem(PoseSystem(self.plant, traj))
            self.robot = self.station.GetSubsystemByName("iiwa_controller").get_multibody_plant_for_control()

            # Set up differential inverse kinematics.
            self.diff_ik = AddIiwaDifferentialIK(builder, self.robot)
            
            builder.Connect(self.diff_ik.get_output_port(), self.station.GetInputPort("iiwa_position"))
            
            builder.Connect(pose_picker.get_output_port(), self.diff_ik.get_input_port(0)) #should this be velocity or position? 
            builder.Connect(self.station.GetOutputPort("iiwa_state_estimated"), self.diff_ik.GetInputPort("robot_state"))
            # ###############################################

        if traj2 is not None:
            """
            This is the gripper trajectory. 
            """
            wsg_position = builder.AddSystem(TrajectorySource(traj2))
            builder.Connect(wsg_position.get_output_port(), self.station.GetInputPort("wsg_position"))
           
        ############## IMPLEMENTING DRAWING ##################
        # self.writer = builder.AddSystem(MeshcatWriter(meshcat,
        #               self.plant.GetBodyByName("cylinder_link").index(),
        #               self.plant.GetBodyByName("paper_link").index(),
        #               Rgba(0, 0, 0, 1.0), line_width=0.003))

        # builder.Connect(self.station.GetOutputPort("contact_results"), self.writer.get_input_port())
        #####################################################

        params = MeshcatVisualizerParams()
        params.delete_on_initialization_event = False
        self.visualizer = MeshcatVisualizer.AddToBuilder(builder, self.station.GetOutputPort("query_object"), meshcat, params)

        self.diagram = builder.Build()
        self.gripper_frame = self.plant.GetFrameByName('body')
        self.world_frame = self.plant.world_frame()
        context = self.CreateDefaultContext()
        
        #################### CAMERA STUFF ########################
        # to_point_cloud = AddPointClouds(
        #     scenario=scenario, station=station, builder=builder, meshcat=meshcat
        # )
        # diagram_context = self.diagram.CreateDefaultContext() #get the context for the entire manipulation station. 
        self.diagram.ForcedPublish(context)
        depth_im_read = self.diagram.GetOutputPort("depth_image").Eval(context).data.squeeze() #change to camera_context 
        self.depth_im = deepcopy(depth_im_read)
        self.depth_im[self.depth_im == np.inf] = 10.0
        self.rgb_im = self.diagram.GetOutputPort("rgb_image").Eval(context).data.squeeze()
        # im ratio
        self.im_ratio = self.rgb_im.shape[0], self.rgb_im.shape[1]

        # Get other info about the camera
        self.cam = self.station.GetSubsystemByName("rgbd_sensor_camera")
        cam_context = self.cam.GetMyMutableContextFromRoot(context)
        self.X_WC = self.cam.body_pose_in_world_output_port().Eval(cam_context)
        self.X_WC = RigidTransform(self.X_WC)
        self.cam_info = self.cam.depth_camera_info()

    def visualize_frame(self, name, X_WF, length=0.15, radius=0.006):
        AddMeshcatTriad(meshcat, "painter/" + name,length=length, radius=radius, X_PT=X_WF)

    def CreateDefaultContext(self):
        context = self.diagram.CreateDefaultContext()
        plant_context = self.diagram.GetMutableSubsystemContext(self.plant, context)
        station_context = self.diagram.GetMutableSubsystemContext(self.station, context)

        # provide initial states
        q0 = np.array([ 1.40666193e-05,  1.56461165e-01, -3.82761069e-05,
                       -1.32296976e+00, -6.29097287e-06,  1.61181157e+00, -2.66900985e-05])
        
        # set the joint positions of the kuka arm
        iiwa = self.plant.GetModelInstanceByName("iiwa")
        self.plant.SetPositions(plant_context, iiwa, q0)
        self.plant.SetVelocities(plant_context, iiwa, np.zeros(7))
        wsg = self.plant.GetModelInstanceByName("wsg")
        self.plant.SetPositions(plant_context, wsg, [-0.005,0.005]) #-0.05, 0.05])
        self.plant.SetVelocities(plant_context, wsg, [0, 0])        

        if hasattr(self, 'integrator'):
            self.integrator.set_integral_value(
                self.integrator.GetMyMutableContextFromRoot(context), q0)

        return context

    def get_X_WG(self, context=None):
        if not context:
            context = self.CreateDefaultContext()
        plant_context = self.plant.GetMyMutableContextFromRoot(context)
        X_WG = self.plant.CalcRelativeTransform(
                    plant_context,
                    frame_A=self.world_frame,
                    frame_B=self.gripper_frame)
        return X_WG

    def paint(self, sim_duration=20.0):
        context = self.CreateDefaultContext()
        simulator = Simulator(self.diagram, context)
        simulator.set_target_realtime_rate(1.0)

        duration = sim_duration if running_as_notebook else 0.01
        simulator.AdvanceTo(duration)

    def project_depth_to_pC(self, depth_pixel):
        v = depth_pixel[:,0]
        u = depth_pixel[:,1]
        Z = depth_pixel[:,2]
        cx = self.cam_info.center_x()
        cy = self.cam_info.center_y()
        fx = self.cam_info.focal_x()
        fy = self.cam_info.focal_y()
        X = (u-cx) * Z/fx
        Y = (v-cy) * Z/fy
        pC = np.c_[X,Y,Z]
        return pC
    
    def visualize_key_frames(self, frame_poses):
        for i, pose in enumerate(frame_poses):
            self.visualize_frame('frame_{}'.format(i), pose, length=0.05)